<a href="https://colab.research.google.com/github/myrondza10/economic_indicators_scoring_fx_calendar/blob/main/Economic_Indicator_Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install ecocal

In [ ]:
from ecocal import *

ec = Calendar(startHorizon="2024-01-01",
                 endHorizon="2024-09-30",
                 withDetails=True,
                 nbThreads=20,
                 preBuildCalendar=True,
                 )

Calendar 	| 2024-01-01 --> 2024-09-30
Number of events: 6620


Details: 100%|██████████████████████████████████████████████████| 6620/6620 [10:27<00:00, 10.55it/s]


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def clean_data(df):
    """
    Clean and preprocess the DataFrame

    :param df: Input DataFrame
    :return: Cleaned DataFrame
    """
    # Convert date column to datetime
    df['datetime'] = pd.to_datetime(df['Start'])

    # Extract month and year
    df['month'] = df['datetime'].dt.to_period('M')

    # Convert numeric columns
    numeric_columns = ['potency', 'volatility']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def filter_by_currency(df, currency):
    """
    Filter events for a specific currency

    :param df: Input DataFrame
    :param currency: Currency code (e.g., 'AUD', 'JPY')
    :return: Filtered DataFrame
    """
    return df[df['Currency'] == currency]

def aggregate_monthly_indicators(df, currency, indicator_column='Name',
                                 aggregation='first'):
    """
    Aggregate monthly indicators for a specific currency

    :param df: Input DataFrame
    :param currency: Currency code
    :param indicator_column: Column to aggregate
    :param aggregation: Aggregation method ('first', 'last', 'mean', etc.)
    :return: Monthly aggregated indicators
    """
    # Filter by currency
    currency_df = filter_by_currency(df, currency)

    # Group by month and aggregate
    monthly_indicators = currency_df.groupby('month')[indicator_column].agg(aggregation)

    return monthly_indicators

def calculate_monthly_volatility(df, currency):
    """
    Calculate monthly volatility for a specific currency

    :param df: Input DataFrame
    :param currency: Currency code
    :return: Monthly volatility DataFrame
    """
    # Filter by currency
    currency_df = filter_by_currency(df, currency)

    # Group by month and calculate volatility metrics
    monthly_volatility = currency_df.groupby('month').agg({
        'volatility': ['mean', 'max', 'min'],
        'potency': ['mean', 'max', 'min'],
        'Name': 'count'  # Number of events
    })

    # Flatten column names
    monthly_volatility.columns = ['_'.join(col).strip() for col in monthly_volatility.columns.values]

    return monthly_volatility

def get_event_summary(df, currency):
    """
    Get summary of economic events for a specific currency

    :param df: Input DataFrame
    :param currency: Currency code
    :return: Summary DataFrame
    """
    # Filter by currency
    currency_df = filter_by_currency(df, currency)

    # Create summary
    summary = pd.DataFrame({
        'total_events': [len(currency_df)],
        'unique_indicators': [currency_df['Name'].nunique()],
        'date_range': [f"{currency_df['datetime'].min()} to {currency_df['datetime'].max()}"],
        'impact_breakdown': [currency_df['Impact'].value_counts().to_dict()]
    })

    return summary

In [ ]:
df = clean_data(ec.detailedCalendar)
df.head(5)

,Id,Start,Name,Impact,Currency,urlSource,comments,description,headline,link,...,isAllDay,isTentative,isPreliminary,isReport,isSpeech,lastUpdated,previousIsPreliminary,categoryId,datetime,month
0,1c25751d-05fa-44d9-aab9-263440e21dcc,01/01/2024 05:00:00,New Year's Day,NONE,CAD,None,None,Banks are closed due to New Year's Day.,None,None,...,True,False,False,False,False,1658238073,False,None,2024-01-01 05:00:00,2024-01
1,eb38981b-e8ec-417a-abfe-5094b49d475b,01/01/2024 05:00:00,New Year's Day,NONE,USD,None,None,Banks are closed due to New Year's Day.,None,None,...,True,False,False,False,False,1686652767,False,None,2024-01-01 05:00:00,2024-01
2,af24421c-609c-42b8-957b-d6fba344f99d,01/01/2024 12:00:00,Day after New Year's Day,NONE,NZD,None,None,Banks will be closed due Day after New Year's ...,None,None,...,True,False,False,False,False,1662634218,False,None,2024-01-01 12:00:00,2024-01
3,3b98fe64-c7b6-455f-8ba4-2bb821bb18e4,01/01/2024 16:00:00,New Years Holiday,NONE,CNY,None,None,Banks will be closed due to New Years Holiday.,None,None,...,True,False,False,False,False,1713368239,False,None,2024-01-01 16:00:00,2024-01
4,e86b297e-15b9-46e9-a028-1914f8a19a8b,01/01/2024 22:00:00,Judo Bank Manufacturing PMI,LOW,AUD,https://www.pmi.spglobal.com/Public/Home/Press...,None,The Manufacturing Purchasing Managers Index (P...,None,None,...,False,False,False,False,False,1729558552,True,None,2024-01-01 22:00:00,2024-01


In [ ]:
df.to_csv('economic_events.csv', index=False)

In [ ]:
df["month"].value_counts()

,count
month,
2024-05,797
2024-04,777
2024-07,735
2024-09,734
2024-03,732
2024-01,718
2024-02,715
2024-08,707
2024-06,705


In [ ]:
def calculate_impact_score(impact):
    """Assign scores to impact levels."""
    impact_mapping = {
        'HIGH': 3,
        'MEDIUM': 2,
        'LOW': 1,
        'NONE': 0
    }
    return impact_mapping.get(impact, 0)

def get_economic_summary(df, currency_pair):
    """
    Get economic summary for a currency pair, aggregated by month.

    :param df: Input DataFrame
    :param currency_pair: Tuple of currencies
    :return: Economic summary DataFrame with month as the first column
    """
    # Ensure the Date column is datetime and create a month column
    df['Date'] = pd.to_datetime(df['Start'])
    df['month'] = df['Date'].dt.to_period('M')

    # Calculate impact_score based on Impact column
    df['impact_score'] = df['Impact'].apply(calculate_impact_score)

    # Calculate weighted_score if not already available (assuming weighted_score = impact_score here as an example)
    if 'weighted_score' not in df.columns:
        df['weighted_score'] = df['impact_score']  # Placeholder logic

    # Filter events for base and quote currencies
    base_currency, quote_currency = currency_pair
    base_events = df[df['Currency'] == base_currency]
    quote_events = df[df['Currency'] == quote_currency]

    # Group by month and calculate monthly metrics
    def calculate_monthly_metrics(events_df):
        monthly = events_df.groupby('month').agg({
            'weighted_score': ['sum', 'mean'],
            'impact_score': ['count', 'max', 'min'],
            'Impact': lambda x: x.value_counts().to_dict()
        })

        # Flatten column names
        monthly.columns = ['total_score', 'avg_score', 'event_count',
                           'max_impact', 'min_impact', 'impact_breakdown']
        return monthly

    # Get monthly metrics for base and quote currencies
    base_monthly = calculate_monthly_metrics(base_events)
    quote_monthly = calculate_monthly_metrics(quote_events)

    # Combine base and quote metrics to calculate relative strength and other details
    combined = pd.DataFrame()
    combined['month'] = base_monthly.index  # Set month as the first column
    combined['base_score'] = base_monthly['total_score'].values
    combined['quote_score'] = quote_monthly['total_score'].values
    combined['relative_strength'] = combined['base_score'] - combined['quote_score']
    combined['score_ratio'] = combined['base_score'] / combined['quote_score']

    # Add detailed metrics
    combined['base_events'] = base_monthly['event_count'].values
    combined['quote_events'] = quote_monthly['event_count'].values
    combined['base_avg_impact'] = base_monthly['avg_score'].values
    combined['quote_avg_impact'] = quote_monthly['avg_score'].values
    combined['base_breakdown'] = base_monthly['impact_breakdown'].values
    combined['quote_breakdown'] = quote_monthly['impact_breakdown'].values

    # Calculate volatility potential
    combined['volatility_index'] = (
        (combined['base_score'] + combined['quote_score']) *
        (combined['base_events'] + combined['quote_events']) / 100
    )

    return combined


In [ ]:
get_economic_summary(df, ('EUR', 'USD'))

,month,base_score,quote_score,relative_strength,score_ratio,base_events,quote_events,base_avg_impact,quote_avg_impact,base_breakdown,quote_breakdown,volatility_index
0,2024-01,377,300,77,1.256667,237,194,1.590717,1.546392,"{'LOW': 125, 'MEDIUM': 78, 'HIGH': 32, 'NONE': 2}","{'LOW': 108, 'MEDIUM': 60, 'HIGH': 24, 'NONE': 2}",2917.87
1,2024-02,319,335,-16,0.952239,201,211,1.587065,1.587678,"{'LOW': 103, 'MEDIUM': 78, 'HIGH': 20}","{'LOW': 105, 'MEDIUM': 85, 'HIGH': 20, 'NONE': 1}",2694.48
2,2024-03,312,348,-36,0.896552,200,214,1.560000,1.626168,"{'LOW': 94, 'MEDIUM': 79, 'HIGH': 20, 'NONE': 7}","{'LOW': 107, 'MEDIUM': 74, 'HIGH': 31, 'NONE': 2}",2732.40
3,2024-04,387,367,20,1.054496,240,225,1.612500,1.631111,"{'LOW': 117, 'MEDIUM': 84, 'HIGH': 34, 'NONE': 5}","{'LOW': 105, 'MEDIUM': 98, 'HIGH': 22}",3506.10
4,2024-05,327,425,-98,0.769412,217,257,1.506912,1.653696,"{'LOW': 115, 'MEDIUM': 79, 'HIGH': 18, 'NONE': 5}","{'MEDIUM': 121, 'LOW': 111, 'HIGH': 24, 'NONE'...",3564.48
5,2024-06,326,350,-24,0.931429,199,215,1.638191,1.627907,"{'MEDIUM': 90, 'LOW': 89, 'HIGH': 19, 'NONE': 1}","{'LOW': 107, 'MEDIUM': 78, 'HIGH': 29, 'NONE': 1}",2798.64
6,2024-07,382,349,33,1.094556,231,216,1.653680,1.615741,"{'LOW': 114, 'MEDIUM': 80, 'HIGH': 36, 'NONE': 1}","{'LOW': 109, 'MEDIUM': 75, 'HIGH': 30, 'NONE': 2}",3267.57
7,2024-08,286,330,-44,0.866667,190,211,1.505263,1.563981,"{'LOW': 103, 'MEDIUM': 66, 'HIGH': 17, 'NONE': 4}","{'LOW': 109, 'MEDIUM': 76, 'HIGH': 23, 'NONE': 3}",2470.16
8,2024-09,318,355,-37,0.895775,200,217,1.590000,1.635945,"{'LOW': 103, 'MEDIUM': 76, 'HIGH': 21}","{'LOW': 108, 'MEDIUM': 77, 'HIGH': 31, 'NONE': 1}",2806.41


In [ ]:
import pandas as pd

# Assuming df is your DataFrame containing economic events

# Base and quote currencies
base_currency, quote_currency = ('EUR', 'USD')

# Filter events for both currencies
base_events = df[df['Currency'] == base_currency].copy()
quote_events = df[df['Currency'] == quote_currency].copy()

# Define category weights
category_weights = {
    'Employment Indicators': 1.5,
    'Manufacturing Indicators': 1.2,
    'Inflation Indicators': 2.0,
    'Consumer Indicators': 1.8,
    'Trade Indicators': 1.0,
    'Production Indicators': 1.3,
    'Housing Indicators': 1.4,
    'Overall Economic Indicators': 2.5,
}

# Categories for top economic indicators
categories = {
    'Employment Indicators': [],
    'Manufacturing Indicators': [],
    'Inflation Indicators': [],
    'Consumer Indicators': [],
    'Trade Indicators': [],
    'Production Indicators': [],
    'Housing Indicators': [],
    'Overall Economic Indicators': [],
}

# Function to categorize events
def categorize_events(events_df):
    for index, row in events_df.iterrows():
        indicator_name = row['Name']
        if 'Job' in indicator_name or 'Employment' in indicator_name:
            categories['Employment Indicators'].append(indicator_name)
        elif 'Manufacturing' in indicator_name:
            categories['Manufacturing Indicators'].append(indicator_name)
        elif 'Price Index' in indicator_name or 'Consumer Price' in indicator_name:
            categories['Inflation Indicators'].append(indicator_name)
        elif 'Consumer' in indicator_name or 'Retail' in indicator_name:
            categories['Consumer Indicators'].append(indicator_name)
        elif 'Trade Balance' in indicator_name:
            categories['Trade Indicators'].append(indicator_name)
        elif 'Production' in indicator_name or 'Industrial' in indicator_name:
            categories['Production Indicators'].append(indicator_name)
        elif 'Housing' in indicator_name or 'Building' in indicator_name:
            categories['Housing Indicators'].append(indicator_name)
        elif 'GDP' in indicator_name or 'Total Vehicle Sales' in indicator_name:
            categories['Overall Economic Indicators'].append(indicator_name)

# Categorize events for both base and quote currencies
categorize_events(base_events)
categorize_events(quote_events)

# Step 1: Add an impact score column to both base and quote events
base_events['impact_score'] = base_events['Impact'].apply(calculate_impact_score)
quote_events['impact_score'] = quote_events['Impact'].apply(calculate_impact_score)

# Step 2: Assign a month column for monthly grouping
base_events['month'] = base_events['Date'].dt.to_period('M')
quote_events['month'] = quote_events['Date'].dt.to_period('M')

# Step 3: Calculate the weighted score for each event based on its category
def assign_weighted_score(events_df):
    weighted_scores = []
    for index, row in events_df.iterrows():
        event_name = row['Name']
        for category, indicators in categories.items():
            if event_name in indicators:
                weighted_score = row['impact_score'] * category_weights[category]
                weighted_scores.append(weighted_score)
                break
        else:
            weighted_scores.append(0)  # Assign default if no category matches
    events_df['weighted_score'] = weighted_scores

# Apply the weighted score assignment
assign_weighted_score(base_events)
assign_weighted_score(quote_events)

# Step 4: Calculate monthly category scores
def calculate_monthly_category_scores(events_df):
    # Initialize monthly scores dictionary
    monthly_scores = {}

    # Group by month
    for month, monthly_events in events_df.groupby('month'):
        category_scores = {category: 0 for category in categories.keys()}

        # Sum scores for each category in the month
        for category, indicators in categories.items():
            for indicator in indicators:
                if indicator in monthly_events['Name'].values:
                    scores = monthly_events.loc[monthly_events['Name'] == indicator, 'weighted_score']
                    category_scores[category] += scores.sum()

        monthly_scores[month] = category_scores

    return monthly_scores

# Get monthly scores for base and quote currencies
base_monthly_scores = calculate_monthly_category_scores(base_events)
quote_monthly_scores = calculate_monthly_category_scores(quote_events)

# Step 5: Display monthly scores
print("*************** MONTHLY CATEGORY SCORES ****************")
for month in base_monthly_scores.keys():
    print(f"\nMonth: {month}")
    for category, base_score in base_monthly_scores[month].items():
        quote_score = quote_monthly_scores[month][category]
        total_score = base_score + quote_score
        print(f"{category}: Base Score = {base_score:.2f}, Quote Score = {quote_score:.2f}, Total = {total_score:.2f}")

# Step 6: Calculate the overall monthly score
print("\n*************** MONTHLY OVERALL SCORES ****************")
for month, category_scores in base_monthly_scores.items():
    total_base_score = sum(base_monthly_scores[month].values())
    total_quote_score = sum(quote_monthly_scores[month].values())
    overall_score = total_base_score + total_quote_score
    print(f"Month: {month} - Overall Score: {overall_score:.2f}")

*************** MONTHLY CATEGORY SCORES ****************

Month: 2024-01
Employment Indicators: Base Score = 0.00, Quote Score = 1161.00, Total = 1161.00
Manufacturing Indicators: Base Score = 1393.20, Quote Score = 237.60, Total = 1630.80
Inflation Indicators: Base Score = 8178.00, Quote Score = 1662.00, Total = 9840.00
Consumer Indicators: Base Score = 1389.60, Quote Score = 554.40, Total = 1944.00
Trade Indicators: Base Score = 98.00, Quote Score = 45.00, Total = 143.00
Production Indicators: Base Score = 224.90, Quote Score = 23.40, Total = 248.30
Housing Indicators: Base Score = 0.00, Quote Score = 88.20, Total = 88.20
Overall Economic Indicators: Base Score = 10.00, Quote Score = 22.50, Total = 32.50

Month: 2024-02
Employment Indicators: Base Score = 27.00, Quote Score = 1251.00, Total = 1278.00
Manufacturing Indicators: Base Score = 1393.20, Quote Score = 237.60, Total = 1630.80
Inflation Indicators: Base Score = 6230.00, Quote Score = 1662.00, Total = 7892.00
Consumer Indicato